In [1]:
import requests
from urllib.error import URLError, HTTPError
from http.client import IncompleteRead
from time import sleep

In [2]:
# kakao api
APP_KEY = ""
kakao_url = "https://dapi.kakao.com/v2/local/search/address.json"
kakao_headers = {"Authorization" : "KakaoAK {}".format(APP_KEY)}

In [3]:
def kakao_req(address):
#     print('HTTP Method: %s' % method)
#     print('Request URL: %s' % kakao_url)
#     print('Headers: %s' % kakao_headers)
#     print('QueryString: %s' % params)
    kakao_url_string = str(kakao_url)+ '?query='+ str(address)
    try:
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    except HTTPError as e:
        print("502 error caused reply" + e)
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    except IncompleteRead:
        # Oh well, reconnect and keep trucking
        print("_chunk_size Error")
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    except URLError as e:
        print(e.reason)
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    else:
        data= resultXML.json()
        if len(data['documents']) ==0:
            data= '403'
            return data
        else:
            return resultXML

In [4]:
def get_location(address):

    params = {"query": "{}".format(address)}

    response = kakao_req(params, 'GET')
        

#    #kakao parsing
#     pprint(data['meta']['total_count'])
#     pprint(data["documents"][0]["x"])
#     pprint(data["documents"][0]["y"])
    
        
    return response

In [5]:
import findspark
findspark.init()
import pyspark
import os
import numpy as np
from pyspark.sql import SQLContext

In [6]:
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [7]:
basic_folder = './office_trade_data/'

file_list = os.listdir(basic_folder)
path_list=[]
for file in file_list:
    print(file)
    path_list.append(basic_folder + file)

office_trade_201701.csv
office_trade_201702.csv
office_trade_201703.csv
office_trade_201704.csv
office_trade_201705.csv
office_trade_201706.csv
office_trade_201707.csv
office_trade_201708.csv
office_trade_201709.csv
office_trade_201710.csv
office_trade_201711.csv
office_trade_201712.csv
office_trade_201801.csv
office_trade_201802.csv
office_trade_201803.csv
office_trade_201804.csv
office_trade_201805.csv
office_trade_201806.csv
office_trade_201807.csv
office_trade_201808.csv
office_trade_201809.csv
office_trade_201810.csv
office_trade_201811.csv
office_trade_201812.csv
office_trade_201901.csv
office_trade_201902.csv
office_trade_201903.csv
office_trade_201904.csv
office_trade_201905.csv
office_trade_201906.csv
office_trade_201907.csv
office_trade_201908.csv
office_trade_201909.csv
office_trade_201910.csv
office_trade_201911.csv
office_trade_201912.csv
office_trade_202001.csv
office_trade_202002.csv
office_trade_202003.csv
office_trade_202004.csv
office_trade_202005.csv
office_trade_202

In [8]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("header", "true").option("inferSchema", "true").load(path_list)

In [9]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- build_year: integer (nullable = true)
 |-- trade_price: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- sub_city: string (nullable = true)
 |-- dong: string (nullable = true)
 |-- code: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- floor: integer (nullable = true)
 |-- exclusive_private_area: double (nullable = true)
 |-- address: string (nullable = true)
 |-- detailed_address: string (nullable = true)



In [10]:
df.show()
df.count()

+---+----------+-----------+----+-----+---+----------+--------+----------+-----+---------------------+-----+----------------------+----------------------------+----------------+
|_c0|build_year|trade_price|year|month|day|      city|sub_city|      dong| code|                 name|floor|exclusive_private_area|                     address|detailed_address|
+---+----------+-----------+----+-----+---+----------+--------+----------+-----+---------------------+-----+----------------------+----------------------------+----------------+
|  0|      2003|     45,800|2020|   12| 28|서울특별시|  종로구|    적선동|11110|      광화문 플래티넘|    4|                 73.76|   서울특별시 종로구  적선동|             156|
|  1|      2003|     61,000|2020|   12| 30|서울특별시|  종로구|    적선동|11110|      광화문 플래티넘|   16|                 73.76|   서울특별시 종로구  적선동|             156|
|  2|      2003|     22,500|2020|   12| 31|서울특별시|  종로구|    적선동|11110|      광화문 플래티넘|   15|                 38.64|   서울특별시 종로구  적선동|             156|
|  3|      2008|   

152458

In [11]:
filter_df = df.na.drop()

In [12]:
from pyspark.sql.functions import *

In [13]:
unique_date_df = filter_df.groupBy(filter_df.year,filter_df.month, filter_df.day).agg(count("day").alias('count'))
# agg = exprs에 들어있는 계산식으로 dataframe 계산

unique_date_list = unique_date_df.collect()
new_date_list = []
for unique_date in unique_date_list:
    
    if unique_date[1] < 10 :
        if unique_date[2] < 10 : 
            date = str(unique_date[0]) + '-' + "0" + str(unique_date[1]) + '-' + "0" +str(unique_date[2]) + 'T' + "00:00:00"
        else:
            date = str(unique_date[0]) + '-' + "0" + str(unique_date[1]) + '-' +str(unique_date[2]) + 'T' + "00:00:00"
        
    else:
        if unique_date[2] < 10 : 
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' + "0" +str(unique_date[2]) + 'T' + "00:00:00"
    
        else:
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' +str(unique_date[2]) + 'T' + "00:00:00"
    
    
    new_date_list.append([unique_date[0], unique_date[1],unique_date[2],date])
    
unique_date_df = sc.parallelize(new_date_list).toDF(['year','month','day','date'])

In [14]:
added_date_df = filter_df.join(unique_date_df, ['year','month','day']).sort('date')
added_date_df.show()
added_date_df.count()

+----+-----+---+---+----------+-----------+----------+--------+-------------+-----+----------------------+-----+----------------------+------------------------------+----------------+-------------------+
|year|month|day|_c0|build_year|trade_price|      city|sub_city|         dong| code|                  name|floor|exclusive_private_area|                       address|detailed_address|               date|
+----+-----+---+---+----------+-----------+----------+--------+-------------+-----+----------------------+-----+----------------------+------------------------------+----------------+-------------------+
|2017|    1|  1|  6|      2007|     38,500|서울특별시|  강동구|       천호동|11740|    대우 한강 베네시티|    5|                 95.26|     서울특별시 강동구  천호동|           425=5|2017-01-01T00:00:00|
|2017|    1|  1|  1|      2016|      9,970|  경상남도|  양산시|물금읍 가촌리|48330|          미건엘리시아|    6|                 31.38| 경상남도 양산시 물금읍 가촌리|          1288=3|2017-01-01T00:00:00|
|2017|    1|  1|  3|      2004|     13,300

145577

In [15]:
added_date_df = added_date_df.withColumn("detailed_address", regexp_replace(col("detailed_address"),"=","-"))
added_date_df.show()

+----+-----+---+---+----------+-----------+----------+--------+-------------+-----+----------------------+-----+----------------------+------------------------------+----------------+-------------------+
|year|month|day|_c0|build_year|trade_price|      city|sub_city|         dong| code|                  name|floor|exclusive_private_area|                       address|detailed_address|               date|
+----+-----+---+---+----------+-----------+----------+--------+-------------+-----+----------------------+-----+----------------------+------------------------------+----------------+-------------------+
|2017|    1|  1|  6|      2007|     38,500|서울특별시|  강동구|       천호동|11740|    대우 한강 베네시티|    5|                 95.26|     서울특별시 강동구  천호동|           425-5|2017-01-01T00:00:00|
|2017|    1|  1|  1|      2016|      9,970|  경상남도|  양산시|물금읍 가촌리|48330|          미건엘리시아|    6|                 31.38| 경상남도 양산시 물금읍 가촌리|          1288-3|2017-01-01T00:00:00|
|2017|    1|  1|  3|      2004|     13,300

In [16]:
new_df = added_date_df.groupBy(added_date_df.address,added_date_df.detailed_address).agg(count("detailed_address").alias("count"))
new_df.show()
new_df.count()

+--------------------------------+----------------+-----+
|                         address|detailed_address|count|
+--------------------------------+----------------+-----+
|       인천광역시 부평구  청천동|          383-14|   52|
|    경기도 수원시 영통구  이의동|            1330|   76|
|      경기도 안산시 상록구  사동|          1197-4|   36|
|       인천광역시 부평구  청천동|           379-8|  279|
|       대전광역시 유성구  지족동|           863-2|   36|
|    경기도 용인시 수지구  상현동|          1133-2|   45|
|     서울특별시 서대문구  창천동|           29-81|   39|
|       서울특별시 강동구  천호동|          167-57|   16|
|서울특별시 영등포구  영등포동6가|              92|    4|
|   경상북도 경산시 하양읍 금락리|          161-32|   80|
|       부산광역시 사하구  괴정동|           555-5|    1|
|     서울특별시 동대문구  이문동|          325-20|   21|
|       서울특별시 은평구  불광동|           319-8|    9|
|       서울특별시 강서구  마곡동|           784-4|   25|
|         서울특별시 도봉구  창동|          632-29|    1|
|             경기도 부천시  상동|           449-1|    1|
|     경기도 평택시 안중읍 안중리|          252-33|   10|
|    경기도 용인시 기흥구  마북동|           

8887

In [17]:
sum = new_df.select(sum("count"))
sum.show()

+----------+
|sum(count)|
+----------+
|    145577|
+----------+



In [18]:
unique_address_list = new_df.collect()
#print (unique_address_list[0][0])

In [19]:
import sys

def req_addr(address):
    data=0
    if '-' in address[1]:
        check = str(address[1]).split('-')[0]
        if address[1].split('-')[0].isdigit() == False or address[1].split('-')[1].isdigit() == False:
            print(address)
            print('주소에 글자섞임')
            data= 400
    if data == 0:
        response = kakao_req(address[0] + " " + address[1])
    else :
        return data
    if response == '403':
        print('빈배열 에러 ' + address[0] + " " + address[1])
        sleep(5)
        data= 3
        return data
    else :
        return response
        
    

In [20]:

location_list = []

# checking why no esult data
# response = get_location("경상북도 영천시 신령면 완전리") -> 신녕면
# print response.status_code
# data = response.json()
# print data

index = 1.0;
length = len(unique_address_list)
count = 0;
print (length)
addr2 =unique_address_list[:1]

for address in unique_address_list: 
    #full_address = (address[0] + " " + address[1]).encode('utf8')
    #print(full_address)
    response = req_addr(address)
    if response == 400 or response == 3:
        continue
    if response.status_code == 200:
        data = response.json()
        x = data['documents'][0]['address']['x']
        y = data['documents'][0]['address']['y']
        location_list.append([address[0],address[1],y,x])
    else: 
        print (response.json())
        count += 1
        #print (count)
        location_list.append([address[0],address[1] ,np.nan,np.nan])
    
    if index % 100 == 0:
        print (("%0.2f" % (index / length * 100.0)) + "% completed")
        
    index += 1.0

8887
빈배열 에러 경기도 용인시 기흥구  구갈동 662
1.13% completed
2.25% completed
빈배열 에러 서울특별시 동대문구  용두동 100-8
3.38% completed
4.50% completed
빈배열 에러 경기도 시흥시  정왕동 1771-24
5.63% completed
6.75% completed
7.88% completed
9.00% completed
10.13% completed
11.25% completed
빈배열 에러 경상남도 양산시  평산동 184
12.38% completed
13.50% completed
14.63% completed
15.75% completed
16.88% completed
18.00% completed
19.13% completed
20.25% completed
21.38% completed
22.50% completed
23.63% completed
24.76% completed
25.88% completed
27.01% completed
28.13% completed
29.26% completed
30.38% completed
31.51% completed
32.63% completed
33.76% completed
34.88% completed
36.01% completed
37.13% completed
빈배열 에러 강원도 원주시  일산동 185-1
38.26% completed
39.38% completed
40.51% completed
빈배열 에러 부산광역시 해운대구  우동 588-1
41.63% completed
42.76% completed
43.88% completed
45.01% completed
46.13% completed
빈배열 에러 경기도 용인시 기흥구  신갈동 42-8
47.26% completed
48.39% completed
49.51% completed
50.64% completed
51.76% completed
52.89% completed
54.01% comp

In [21]:
location = sc.parallelize(location_list).toDF(['address','detailed_address','latitude','longitude'])
location.show()

+--------------------------------+----------------+----------------+----------------+
|                         address|detailed_address|        latitude|       longitude|
+--------------------------------+----------------+----------------+----------------+
|       인천광역시 부평구  청천동|          383-14|37.5226833421997|126.700932681554|
|    경기도 수원시 영통구  이의동|            1330|37.2925283683075|127.049160071729|
|      경기도 안산시 상록구  사동|          1197-4|37.3017037893824|126.851100920471|
|       인천광역시 부평구  청천동|           379-8| 37.523156953968|126.699885410668|
|       대전광역시 유성구  지족동|           863-2|36.3825806256824|127.320554874073|
|    경기도 용인시 수지구  상현동|          1133-2|37.2959868892725|127.069970873404|
|     서울특별시 서대문구  창천동|           29-81|37.5560565935819|126.938257661301|
|       서울특별시 강동구  천호동|          167-57|37.5388281813469|127.132318119245|
|서울특별시 영등포구  영등포동6가|              92|  37.52090651092| 126.90180167613|
|   경상북도 경산시 하양읍 금락리|          161-32|35.9079799653822|128.812900975347|


In [22]:
location.count()

8869

In [23]:
joined_df = added_date_df.join(location.select('address','detailed_address','latitude','longitude'), ['address','detailed_address'])

In [24]:
joined_df.show()
joined_df.count()

+----------------------------+----------------+----+-----+---+---+----------+-----------+------+-------------+-------+-----+------------------------+-----+----------------------+-------------------+----------------+----------------+
|                     address|detailed_address|year|month|day|_c0|build_year|trade_price|  city|     sub_city|   dong| code|                    name|floor|exclusive_private_area|               date|        latitude|       longitude|
+----------------------------+----------------+----+-----+---+---+----------+-----------+------+-------------+-------+-----+------------------------+-----+----------------------+-------------------+----------------+----------------+
|         경기도 부천시  상동|           449-1|2018|   11| 12| 26|      1996|     10,000|경기도|       부천시|   상동|41190|          동아하이츠빌딩|    8|                 62.74|2018-11-12T00:00:00|37.4899890091982|126.756519978653|
|       경기도 부천시  심곡동|          373-11|2017|    5| 13|  4|      2014|     18,000|경기도|       

145425

In [25]:
drop_index_df = joined_df.drop("_c0")
drop_index_df.show()
drop_index_df.count()

+----------------------------+----------------+----+-----+---+----------+-----------+------+-------------+-------+-----+------------------------+-----+----------------------+-------------------+----------------+----------------+
|                     address|detailed_address|year|month|day|build_year|trade_price|  city|     sub_city|   dong| code|                    name|floor|exclusive_private_area|               date|        latitude|       longitude|
+----------------------------+----------------+----+-----+---+----------+-----------+------+-------------+-------+-----+------------------------+-----+----------------------+-------------------+----------------+----------------+
|         경기도 부천시  상동|           449-1|2018|   11| 12|      1996|     10,000|경기도|       부천시|   상동|41190|          동아하이츠빌딩|    8|                 62.74|2018-11-12T00:00:00|37.4899890091982|126.756519978653|
|       경기도 부천시  심곡동|          373-11|2017|    5| 13|      2014|     18,000|경기도|       부천시| 심곡동|41190|     

145425

In [26]:
final_apartment_df = drop_index_df.na.drop()
final_apartment_df = final_apartment_df.withColumn('trade_price',regexp_replace('trade_price',"\\,", ""))
final_apartment_df.show()
final_apartment_df.count()

+----------------------------+----------------+----+-----+---+----------+-----------+------+-------------+-------+-----+------------------------+-----+----------------------+-------------------+----------------+----------------+
|                     address|detailed_address|year|month|day|build_year|trade_price|  city|     sub_city|   dong| code|                    name|floor|exclusive_private_area|               date|        latitude|       longitude|
+----------------------------+----------------+----+-----+---+----------+-----------+------+-------------+-------+-----+------------------------+-----+----------------------+-------------------+----------------+----------------+
|         경기도 부천시  상동|           449-1|2018|   11| 12|      1996|      10000|경기도|       부천시|   상동|41190|          동아하이츠빌딩|    8|                 62.74|2018-11-12T00:00:00|37.4899890091982|126.756519978653|
|       경기도 부천시  심곡동|          373-11|2017|    5| 13|      2014|      18000|경기도|       부천시| 심곡동|41190|     

145425

In [27]:
#final_apartment_df.write.format('com.databricks.spark.csv').save('mycsv.csv')
final_apartment_df.coalesce(1).write.csv('office_trade.csv')